# Hadoop Streaming assignment 0: Word Count

Push 'Open Tool' and get access to your first notebook with the WordCount MapReduce job. Open the notebook, run the job, check that it is successful and push 'Submit assignment'.

In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Writing reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [4]:
! hdfs dfs -ls /data/wiki

Found 4 items
drwxr-xr-x   - hdfs supergroup          0 2017-07-03 23:21 /data/wiki/en_articles
drwxr-xr-x   - hdfs supergroup          0 2017-07-03 22:37 /data/wiki/en_articles_part
drwxr-xr-x   - hdfs supergroup          0 2017-08-11 04:12 /data/wiki/en_articles_part1
-rw-r--r--   2 hdfs supergroup       1914 2017-08-04 16:12 /data/wiki/stop_words_en-xpo6.txt


In [5]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

0%however	1
0&\mathrm{if	1
0(8)320-1234	1
0)).(1	2
0,03	1
0,1,...,n	1
0,1,0	1
0,1,\dots,n	1
0,5	1
0,50	1


rm: `wordcount_result_1504600278440756': No such file or directory
17/09/05 12:31:23 INFO client.RMProxy: Connecting to ResourceManager at mipt-master.atp-fivt.org/93.175.29.106:8032
17/09/05 12:31:23 INFO client.RMProxy: Connecting to ResourceManager at mipt-master.atp-fivt.org/93.175.29.106:8032
17/09/05 12:31:24 INFO mapred.FileInputFormat: Total input paths to process : 1
17/09/05 12:31:24 INFO mapreduce.JobSubmitter: number of splits:3
17/09/05 12:31:24 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1503302131685_0379
17/09/05 12:31:25 INFO impl.YarnClientImpl: Submitted application application_1503302131685_0379
17/09/05 12:31:25 INFO mapreduce.Job: The url to track the job: http://mipt-master.atp-fivt.org:8088/proxy/application_1503302131685_0379/
17/09/05 12:31:25 INFO mapreduce.Job: Running job: job_1503302131685_0379
17/09/05 12:31:31 INFO mapreduce.Job: Job job_1503302131685_0379 running in uber mode : false
17/09/05 12:31:31 INFO mapreduce.Job:  map 0% reduce 0